In [4]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [7]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

Creating an instance of our model

In [8]:
lr = LinearRegression(featuresCol='features',
                     labelCol='label',
                     predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.r2

0.027839179518600154

In [14]:
training_summary.rootMeanSquaredError

10.16309157133015

We didn't split the data into train and testing. So we will do that now.

In [15]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [23]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [26]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                359|
|   mean|0.03148968422227796|
| stddev| 10.115220532857437|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [27]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                142|
|   mean| 0.8267359386189599|
| stddev| 10.829197374722854|
|    min|-28.571478869743427|
|    max| 24.290551295953957|
+-------+-------------------+



In [28]:
correct_model = lr.fit(train_data)

In [30]:
test_results = correct_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  -26.9225858465206|
| -27.71884362755675|
|-23.120131283198916|
| -21.05566378525206|
| -21.26185699192666|
|-20.646787732230422|
|-21.428485332476573|
|-20.198053525720038|
|-17.240351993919834|
|-15.349968920496242|
|-16.871852709893247|
|-17.683184574142487|
|-15.962605189197511|
|-17.182307744699486|
|-13.093255171754308|
| -9.169936535892859|
|-13.869184041606594|
| -12.20095646096373|
|-10.088554985805882|
| -9.885213487434985|
+-------------------+
only showing top 20 rows



In [32]:
test_results.rootMeanSquaredError

10.93961686679461

In [33]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [34]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-1.6488930232228247|
|(10,[0,1,2,3,4,5,...|-0.3271744102198806|
|(10,[0,1,2,3,4,5,...| 2.9080540242402435|
|(10,[0,1,2,3,4,5,...| 1.1711030109786356|
|(10,[0,1,2,3,4,5,...| 1.4790942023121216|
|(10,[0,1,2,3,4,5,...| 0.9794691168587012|
|(10,[0,1,2,3,4,5,...| 2.0261493022620214|
|(10,[0,1,2,3,4,5,...| 2.7693789547805308|
|(10,[0,1,2,3,4,5,...|0.17495236804381728|
|(10,[0,1,2,3,4,5,...|-1.6765233437133065|
|(10,[0,1,2,3,4,5,...| 0.1796456885821424|
|(10,[0,1,2,3,4,5,...| 1.9596689610939182|
|(10,[0,1,2,3,4,5,...| 1.1404522794463217|
|(10,[0,1,2,3,4,5,...|  2.853329235624042|
|(10,[0,1,2,3,4,5,...| -0.327339604136449|
|(10,[0,1,2,3,4,5,...|-3.6032904633583382|
|(10,[0,1,2,3,4,5,...| 1.3777419640601822|
|(10,[0,1,2,3,4,5,...|-0.2783237504877658|
|(10,[0,1,2,3,4,5,...|-2.3223914170342783|
|(10,[0,1,2,3,4,5,...|-2.2451397248529448|
+----------